In [ ]:
!apt-get -qq update
!apt-get -qq install postgresql postgresql-client postgresql-contrib
!pip install psycopg2

In [ ]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [ ]:
!sudo -u postgres psql -c "CREATE USER adminuser WITH PASSWORD 'admin123';"
!sudo -u postgres psql -c "CREATE DATABASE sportstickets;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE sportstickets to adminuser;"
!sudo -u postgres psql -c "GRANT pg_read_server_files TO adminuser;"
!sudo -u postgres psql -c "CREATE SCHEMA IF NOT exists dms_sample;"

CREATE ROLE
CREATE DATABASE
GRANT
GRANT ROLE
CREATE SCHEMA


In [ ]:
!sudo sed -i "s/#listen_addresses = 'localhost'/listen_addresses = '*'/g" /etc/postgresql/14/main/postgresql.conf
!sudo echo "host sportstickets adminuser 0.0.0.0/0 md5" >> /etc/postgresql/14/main/pg_hba.conf

In [ ]:
! cat /etc/postgresql/14/main/postgresql.conf

In [ ]:
! cat /etc/postgresql/14/main/pg_hba.conf

In [ ]:
!service postgresql restart

 * Restarting PostgreSQL 14 database server
   ...done.


In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xf ngrok-v3-stable-linux-amd64.tgz

In [ ]:
!./ngrok authtoken YOUR TOKEN NGROK

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
get_ipython().system_raw('./ngrok tcp 5432 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

tcp://0.tcp.ngrok.io:14732


In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Replace these values with your own database connection details
jdbc_url = "jdbc:postgresql://YOUR_LINK_NGROK:YOUR_PORT_NGROK/sportstickets"
# Example "jdbc:postgresql://0.tcp.ngrok.io:14732/sportstickets"

# Extracting values from the JDBC URL
url_parts = jdbc_url.split("/")
dbname = url_parts[-1]
host_and_port = url_parts[-2].split(":")
host = host_and_port[0]
port = host_and_port[1]

# Add your username and password if required
user = "adminuser"
password = "admin123"

# Connect to the PostgreSQL server
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

# Set isolation level to autocommit to create a new schema
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Replace 'dms_sample' with your desired schema name
schema_name = 'dms_sample'

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create the schema
create_schema_query = sql.SQL("CREATE SCHEMA IF NOT EXISTS {};").format(sql.Identifier(schema_name))
cursor.execute(create_schema_query)

# Read the SQL script file
with open('query_create_tables.sql', 'r') as file:
    sql_script = file.read()

# Execute the SQL script
cursor.execute(sql_script)

# Read the SQL script file
with open('load_data.sql', 'r') as file:
    load_script = file.read()

# Execute the SQL script
cursor.execute(load_script)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()